# DeeplabV3+ Semantic Segmentation Keras Model - Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/imx500_notebooks/keras/keras_deeplabv3plus_for_imx500.ipynb)

## Overview

In this tutorial, we'll quantize the DeeplabV3+ model for semantic segmentation with MCT's post-training quantization techniques. The pretrained model was created with [bonlime's repo](https://github.com/bonlime/keras-deeplab-v3-plus), after making the following changes:

1. Input image size set to 320x320.
2. `expand_dims` in `Lambda` layer replaced with a simple `Reshape` layer.
3. `tf.compat.v1.image.resize` in `Lambda` layer replaced with a `Resizing` layer.
4. Added `argmax` at model output to calculate the class id.

## Setup
### Install the relevant packages

In [ ]:
TF_VER = '2.14.0'

!pip install -q tensorflow=={TF_VER}
!pip install 'huggingface-hub<=0.21.4'

import importlib

if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit

### Download Pascel VOC 2012 dataset

Download the Pascal dataset to the local folder.

In [ ]:
import os

if not os.path.isdir('VOCdevkit'):
    !wget -nc http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
    !tar -xf VOCtrainval_11-May-2012.tar
    !echo Done loading PascelVOC 2012

## Model Quantization

### Download a Pre-Trained Model 

We begin by loading a pre-trained [DeeplabV3+](https://huggingface.co/SSI-DNN/keras_deeplabv3_plus_320) model.   

In [ ]:
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras('SSI-DNN/keras_deeplabv3_plus_320')

### Create dataset object

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2


class PascalDataset:
    def __init__(self, img_size=320, batch_size=16):
        base_path = 'VOCdevkit/VOC2012'
        val_file = os.path.join(base_path, 'ImageSets', 'Segmentation', 'val.txt')
        with open(val_file) as f:
            self.images_names = [os.path.join(base_path, 'JPEGImages', fname.split('\n')[0] + '.jpg')
                                 for fname in f.readlines()]
        self.annotations_dict = {}
        with open(val_file) as f:
            for fname in f.readlines():
                full_path_label_file = os.path.join(base_path, 'SegmentationClass', fname.split('\n')[0] + '.png')
                self.annotations_dict.update({os.path.basename(full_path_label_file.replace('.png', '.jpg')): full_path_label_file})

        self.inds = list(range(len(self.images_names)))
        self.img_size = img_size
        self.batch_size = batch_size

    def shuffle(self):
        self.inds = np.random.permutation(self.inds)

    def __len__(self):
        return int(np.ceil(len(self.images_names) / self.batch_size))

    def __iter__(self):
        img_batch, ann_batch = [], []
        for b, i in enumerate(self.inds):
            img_name = self.images_names[i]
            _name = img_name.split('/')[-1]
            img = np.array(Image.open(img_name))
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = (img - 127.5) / 127.5
            ann = np.array(Image.open(self.annotations_dict[_name]))
            ann = cv2.resize(ann, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)

            img_batch.append(img)
            ann_batch.append(ann)
            if len(img_batch) == self.batch_size:
                yield [np.stack(img_batch), np.stack(ann_batch)]
                img_batch, ann_batch = [], []

        yield [np.stack(img_batch), np.stack(ann_batch)]

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 320 images, divided into 20 iterations of 'batch_size' images each. 

In [ ]:
import model_compression_toolkit as mct
from typing import Iterator, List


n_iters = 20


# Define representative dataset generator
def get_representative_dataset():
    """
    This function creates a representative dataset generator. The generator yields numpy
    arrays of batches of shape: [Batch, H, W ,C].
    
    Returns:
        A representative dataset generator
    """       

    representative_dataset = PascalDataset()
    representative_dataset.shuffle()
    
    
    def _representative_dataset() -> Iterator[List]:
        ds_iter = iter(representative_dataset)
        for _ in range(n_iters):
            yield [next(ds_iter)[0]]

    return _representative_dataset


# Set IMX500-v1 TPC
tpc = mct.get_target_platform_capabilities("tensorflow", 'imx500', target_platform_version='v1')

# Perform post training quantization
quant_model, _ = mct.ptq.keras_post_training_quantization(model,
                                                          get_representative_dataset(),
                                                          target_platform_capabilities=tpc)
print('Quantized model is ready')

### Model Export

Now, we can export the quantized model, ready for deployment, into a `.keras` format file. Please ensure that the `save_model_path` has been set correctly. 

In [ ]:
mct.exporter.keras_export_model(model=quant_model, save_model_path='qmodel.keras')

## Evaluation on Pascal dataset

### Floating point model evaluation

Evaluate the floating point model on PascalVoc using tensorflow MeanIoU metric.

In [ ]:
import tensorflow as tf
from tqdm import tqdm

metric = tf.keras.metrics.MeanIoU(21, ignore_class=255)
for imgs, labels in tqdm(PascalDataset()):
    out = model(imgs)
    metric.update_state(labels, out)
print(f'\nFloat model MeanIOU = {metric.result().numpy()*100:2.3f}')

### Quantized model evaluation
Lastly, we can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
metric = tf.keras.metrics.MeanIoU(21, ignore_class=255)
for imgs, labels in tqdm(PascalDataset()):
    out = quant_model(imgs)
    metric.update_state(labels, out)
print(f'\nQuantized model MeanIOU = {metric.result().numpy()*100:2.3f}')

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.